In [65]:
from collections import *
import math
import numpy as np
from subprocess import Popen, PIPE

letterindex = {'A': 0, 'a': 0, 'T': 1, 't': 1, 'C': 2, 'c': 2, 'G': 3, 'g': 3, 'N': -1, 'n': -1}

def bases(chrom, start, end):
    seq_count = int(math.ceil((float(end - start) / 60.0)))
    sum_seq = ""
    for i in xrange(seq_count - 1):
        p = Popen(['samtools', 'faidx', '../Genome/hg19.fa', 'chr' + str(chrom) + ':' + str(start + 1 + i * 60) + '-' + str(start + 1 + (i + 1) * 60)], stdin=PIPE, stdout=PIPE, stderr=PIPE)
        output, err = p.communicate()
        sum_seq = sum_seq + output.split('\n')[1]
    p = Popen(['samtools', 'faidx', '../Genome/hg19.fa', 'chr' + str(chrom) + ':' + str(start + 1 + (seq_count - 1) * 60) + '-' + str(end)], stdin=PIPE, stdout=PIPE, stderr=PIPE)
    output, err = p.communicate()
    sum_seq = sum_seq + output.split('\n')[1]
    return sum_seq

In [69]:
cell_types =  ['HepG2', 'K562']
promoters = ['SV40P', 'minP']
design_names = ['ScaleUpDesign1', 'ScaleUpDesign2']

coords_to_base = {}
id_to_seq = {}

no_0 = ['Hepg2_5_511_chr20_32276995', 'H1hesc_1_112_chr17_38804455', 'Hepg2_5_1164_chr3_129209275', 'H1hesc_5_471_chr16_17930955', 'Hepg2_5_346_chr17_9588755']
no_1 = ['K562_9_33_chr8_68983455', 'K562_12_4_chr16_9127475', 'H1hesc_5_942_chr18_47655295', 'Huvec_8_469_chr15_57633055']
no_30 = ['Huvec_8_284_chr2_191917035', 'Hepg2_12_0_chr3_169480875', 'Hepg2_5_102_chr4_100065535', 'H1hesc_8_474_chr9_96764975', 'Hepg2_4_10_chr16_81478415']
all_bad = no_0 + no_1 + no_30

for design in design_names:
    with open("../data/Scaleup_counts_sequences/" + design + ".sequences.txt") as f:
        current = "H1hesc_1_100_chr7_11013415"
        counter = []
        for line in f:
            params = line.strip().split()
            params_parse = params[0].split('_')
            line_id = '_'.join(params_parse[0:3] + params_parse[4:])
            current_count = params_parse[3]
            if current_count == '0':
                id_to_seq[line_id] = params[1]
            elif current_count == '1':
                if line_id in id_to_seq:
                    id_to_seq[line_id] = id_to_seq[line_id] + params[1][-5:]
            elif current_count == '29':
                if line_id in no_1:
                    if line_id in id_to_seq:
                        id_to_seq[line_id] = id_to_seq[line_id] + params[1][:5]
            elif current_count == '30':
                if line_id in id_to_seq:
                    id_to_seq[line_id] = id_to_seq[line_id] + params[1]
            if line_id != current:
                current = line_id

id_to_seq["Hepg2_5_511_chr20_32276995"] = 'GTACCTATTAATACCTACTTGAAAGCAGAAAAGCCTGATAGATGAGGCCCCTTGGCCCTAAGCCTGGCAGCTCCTGATAAGGCTGTTCTCAATGTCAACAGTTTTCAAAAAACAGTTTCCAAAAAATCCAcagtcactgggcagccacttccattgatgatctcctttcaaccccataatcccgtaagtaagcttgtttatccacatggtgcaggaaggaagaggctcagagaggggcattcatagctcaaggtcacacagcagaactgagatgagaacttcacttgtcccactg'.upper()
id_to_seq["H1hesc_1_112_chr17_38804455"] = 'GCCGAGGAGGCCCGACCTAACACAGAATAGGCCTGGGTTTAGCGGCTGTGAGGAGCCGGAGCCGCCGCAGCGACCGAGGAAGTGGCGGGAGGTGCCGAGCCTTGCGGAGGGGCGGTGGGGGCCGCGGTGGGAGACGGCGGATGGCGCTCAGCGCGAGGCCGCGGAACCACTGTGGCAGCAAGCAGTTCGGCCAGACTCCCGCTGAGCCCGCTCCTGGAGGGGCCCTCCCCGACTCCGGTCTTGCTTTCCTTTAGGCTGTGGGTCAGAGTCTACACTTCACTAAACCTGGGATGGA'.upper()
id_to_seq["Hepg2_5_1164_chr3_129209275"] = 'GTACCCCACCTCTTTACTCTGTGCCAGGCACTCAGGCTCACCAGCTTGGATATAGTGGTTCCTAGAGATGGACAGCCATCCCCACTAGATGGCAGAGAAGAGCTGCTGAGCTCCCGCCAGACCCTGAGCAGAGCTGCAGGTTCCTCCCTATTCCCAGGCTTCCCTCTGCTGAGCTCATTCAGGGGCCTTGTGACTCTGACCAGGCACCCTGACGCTGCAAAATGTGTCAGGTTGAAGGAACTCGGAAACAGATTTATTAATGGGCAAATCCTAGACACTAGCACAGCTGAGCTAC'.upper()
id_to_seq["H1hesc_5_471_chr16_17930955"] = 'agtaccgcccaatcaattgtgagtagaagtcttccgtgtcacttctggctaatgcgttcatgagccactgcaagaccctccaacagtcccttttgttgtggcagcaacctggaagccatgtgatcaaagcagcttacatttctgagtcatcacctaaaagacggtcctatagagttgcctaaccagctttggaatttgcgtgagtgatacgtaagcctttgtcatgctaaaccactaacatttctgggttagtctgttcttacagcacagcctagattaccctgattaaACGCAT'.upper()
id_to_seq["Hepg2_5_346_chr17_9588755"] = 'GTACCCACGGCAGAGTTAGGGCCCATAGGCGCAGGCTCACCCAGCATTCTGTACACCATGAGCTCACTGCTGTGCGCTTACTATGTTGGGACATGGATACCATCCCAGTGATGAGAATATTCTCGATACCCCAACCAAAGTTGTGTCATCTGGATTTATGTCTCTGCCTTGCCTTCCCCTGGCTCCAAGCTTGTCTTACTCTATTCCCTGCTGTGAGTAAGAGATTTTGCCAGCTAGCTGGTCATTTTGAGTAATCTGTTTACACAGCAGCATCTACCTGGCTTTGTGTCACTGG'.upper()

id_to_seq["Huvec_8_284_chr2_191917035"] = 'TTCCTCACTCTTTCCTAACCTTTGTGTGATCATCCCTACTGAATGGCCAAGCCCCTGATTCCCGGTTTATCTTCCTTCATGTACTTCATGACTAGTCAACTGTTACTGCTGCACACAAGACACGGTTCAATCTGTGGTTTGTTTTTAAATGTACCTAGGAGTCATACTAAAATTCTGCCAACAGGTGGAGCTCAAAGAATAGCTTAAAATAAAATTAACCCGATTCGGCAAACCTGGAAAAAAAGGGAAGAAATAAAATCATTCTGATACGACTGTATGATTTTGCTATTCATCA'.upper()
id_to_seq["Hepg2_12_0_chr3_169480875"] = 'CTAGAAAACAACTATGGACTTATTTTTAAATTGAATGTATTAAATCACTGTTTTCTTTTTAAAAATGAAAGAAAAACATTCCCAGTCTGTGGCCATTCTTGCTTCACGGCCCTGCACCACCACAAATGTTGTAAATGTGGAGAAGCAAAAGTACCACTAGATGGAGTAAGAACAAAAGACATGCACTTGTCTGTAGTTCAAGGAGTCCCCGCCCTTGCAAAAGGGATCTTAGTCCCCgcacggtggctcacgcctgtaatcccagcactttgggaggcagaagcgggcggatcac'.upper()
id_to_seq["Hepg2_5_102_chr4_100065535"] = 'TGTGTTGGAAGTTTCTTTCTGAGTTAAGAAAGCTTCAAACTCCTACCCAGGGAGTTCCGTGTCCTATAATGAGCTGTCTTTGAATAAATACCATTCATGTTGCTTAATTCTCCAATCACTTCCTTTTCTCCTTTGTTGCTGTAATTCACAGCCCTGCCGGCCTCTTACCACATCATTTGTACTATTGGACTTCAGCTGCAAACCTCATTGCTCCCAGTGAGCTCCAAGattttttaaattcttttttcttttattgaaaaatataaaataataGAGAAAAGCCCATTAAACATAT'.upper()
id_to_seq["H1hesc_8_474_chr9_96764975"] = 'gaggactgaagcaaagtggcctctaggcttgacccaaggcagcacggttggcaagggacgacagactctgggtgcatggctggggtgcagtgccactgggtgtaactagaggaagtgcaaaagcaagaaaaggcagaacgcagcacaggagtaccaagaagagcccccttcttattgcaatgtttctctagcgccctctgctgacaatgcttaacattttgctcactgtaaaagggaaatccaggccactaatatggaaaaggtactcttagagcaaatccacagctatgaggca'.upper()
id_to_seq["Hepg2_4_10_chr16_81478415"] = 'CGCCCGCGAGAGGCCGAAGCCGCCGCGGCCGCCGCCGTTGCCGCCATTGACGTCAGAGGGGCGGGCACATGACCCTGCGGGGACCAATcgcgcgggcgccccgggctccgggagcccggcgcgcggcccgccccgcccgcggccACTCTCGCCCGGACGGCCGCGCGGACACACGCTCTGTACACAcgcgcgcggcggcgcggggccccgAGACACGcccgcccccaccccgccgcccccaccccggcgcccgccctccgcgcctggccccggcccgccctcggcctcccccgcc'.upper()

for design in design_names:
    with open("../data/Scaleup_counts_sequences/coords_" + design + "_hg19.txt") as f:
        for line in f:
            params = line.strip().split()
            id_to_seq[params[0]] = (id_to_seq[params[0]], (params[1][3:], int(params[2]), int(params[3])))

In [6]:
from mrpa_data import MrpaData

seq_dict = MrpaData().split_data

[('HepG2', 'minP'), ('K562', 'minP'), ('HepG2', 'SV40P'), ('K562', 'SV40P')]

In [70]:
id_to_seq["H1hesc_1_59_chr4_170192915"]

('CGGAGCGAAGTGTTGCCCCTTAGGGCCGAGAGCGGCGTGGTGAATCGATGGGGACAGTGCCGCGGCCACCTGTCCGCACCCAGCCATCAGTCACCCCAGGACCCAAACCGGAGCAAGCCTGGAGGACCGCTGACCCCAAAGCCCAAGACTAGGAAAACGGTAGTACGGACTTCCAACCCAGGCTCGGGGGCTCTGTTATCCCTGCCCTAAGTGGGGAGAATTGAGTCTGACCTCAGGTGGGCTGGCACCGCAAAGCACGCTGAGAGGCGCCTCTTTTATCTGAGCCTGGGTGGCT',
 ('4', 170192768, 170193063))

In [78]:
import json
with open("id_dict.txt", 'w') as f:
    f.write(json.dumps(id_to_seq))